In [1]:
#finding the best validation accuracy
#with handcoded features
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from torch.utils.data import Subset
import wandb
from PIL import Image
from tabulate import tabulate
import numpy as np
import random
import shutil

In [2]:


# Load just color and texture
color_df   = pd.read_csv('/kaggle/input/color-features/color_features_full.csv')
texture_df = pd.read_csv('/kaggle/input/texture-features/glcm_ngtdm_features_full.csv')

# Drop any “Class” column
for df in (color_df, texture_df):
    if 'Class' in df.columns:
        df.drop(columns=['Class'], inplace=True)

#  Merge on Image_ID and set index
features_df = (
    color_df
    .merge(texture_df, on='Image_ID')
    .set_index('Image_ID')
)

# 4) Collapse duplicates if any
features_df = features_df.groupby(level=0).first()

# 5) Keep only numeric columns
features_df = features_df.select_dtypes(include=[np.number])

# 6) Record feature dimension
feature_dim = features_df.shape[1]

In [3]:
#  CUSTOM DATASET  RETURNING (img, tab_feats, label) 
class SkinDataset(Dataset):
    def __init__(self, root, transform, features_df):
        self.folder   = datasets.ImageFolder(root=root, transform=transform)
        self.features = features_df

    def __len__(self):
        return len(self.folder)

    def __getitem__(self, idx):
        #load image+label
        img, label = self.folder[idx]

        # extract Image_ID from the filepath
        path, _   = self.folder.samples[idx]
        image_id  = os.path.splitext(os.path.basename(path))[0]

        # look up and convert the numeric features
        row = self.features.loc[image_id]              
        tab = torch.tensor(row.values, dtype=torch.float32)

        # return the triplet
        return img, tab, label


In [4]:
# TRANSFORMS & DATASET SPLITS 
IMG_SIZE = (224, 224)
transform_pipeline = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

image_root  = '/kaggle/input/preprocessed-data/Optimized_output_final'
test_folder = '/kaggle/working/opt_output_test2'
random.seed(42)
os.makedirs(test_folder, exist_ok=True)

# Build your full SkinDataset (as before)
dataset = SkinDataset(
    root=image_root,
    transform=transform_pipeline,
    features_df=features_df
)

# Stratified per-class sampling for test set
test_paths = set()
for cls in sorted(os.listdir(image_root)):
    src_dir = os.path.join(image_root, cls)
    if not os.path.isdir(src_dir):
        continue

    # gather all images for this class
    imgs = [f for f in os.listdir(src_dir)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    # sample 20%
    n_sample = max(1, int(0.2 * len(imgs)))
    sampled = random.sample(imgs, n_sample)

    # copy each sampled file and record its source path
    dst_dir = os.path.join(test_folder, cls)
    os.makedirs(dst_dir, exist_ok=True)
    for fn in sampled:
        src_path = os.path.join(src_dir, fn)
        shutil.copy2(src_path, os.path.join(dst_dir, fn))
        test_paths.add(src_path)

#  Build train/val splits from the remaining images
all_indices = list(range(len(dataset)))
rest_indices = [
    idx for idx in all_indices
    if dataset.folder.samples[idx][0] not in test_paths
]

n_trainval = len(rest_indices)
n_val      = int(0.1 * n_trainval)
n_train    = n_trainval - n_val

random.shuffle(rest_indices)
train_idxs = rest_indices[:n_train]
val_idxs   = rest_indices[n_train:]

#  Create in-memory Subsets for train & val
train_ds = Subset(dataset, train_idxs)
val_ds   = Subset(dataset, val_idxs)

#  Build test_ds pointing at the on-disk test_folder
test_ds = SkinDataset(
    root=test_folder,
    transform=transform_pipeline,
    features_df=features_df
)
# ZIP THE TEST FOLDER FOR DOWNLOAD ───
zip_base = '/kaggle/working/opt_output_test2'   # no “.zip” here
shutil.make_archive(zip_base, 'zip', test_folder)
print("Zipped test folder to:", zip_base + '.zip')

Zipped test folder to: /kaggle/working/opt_output_test2.zip


In [6]:
# PRINT DATA DETAILS 
from tabulate import tabulate

# recompute the counts
total   = len(dataset)
n_train = len(train_idxs)
n_val   = len(val_idxs)
n_test  = len(test_ds)

# class counts in your preprocessed image folder
class_counts = []
for class_name in sorted(os.listdir(image_root)):
    path = os.path.join(image_root, class_name)
    if os.path.isdir(path):
        count = len([
            f for f in os.listdir(path)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ])
        class_counts.append((class_name, count))

print(tabulate(class_counts, headers=['Class Name', '# Images']))
print(f"\nTotal images:      {total}")
print(f"  Train images:    {n_train}")
print(f"  Val images:      {n_val}")
print(f"  Test images:     {n_test}\n")


Class Name                                                   # Images
---------------------------------------------------------  ----------
1. Eczema                                                        1677
10. Warts Molluscum and other Viral Infections                   2103
2. Melanoma                                                      3140
3. Atopic Dermatitis                                             1257
4. Basal Cell Carcinoma                                          3323
5. Melanocytic Nevi                                              7970
6. Benign Keratosis-like Lesions                                 2079
7. Psoriasis pictures Lichen Planus and related diseases         2055
8. Seborrheic Keratoses and other Benign Tumors                  1847
9. Tinea Ringworm Candidiasis and other Fungal Infections        1702

Total images:      27153
  Train images:    19554
  Val images:      2172
  Test images:     5427



In [7]:
# EXTEND ResNet TO CONCAT TABULAR FEATURES
class ResNetWithTabular(nn.Module):
    def __init__(self, base_model, tab_dim, num_classes=10):
        super().__init__()
        # everything except the final fc
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])  # output: [B,2048,1,1]
        self.tab_bn   = nn.BatchNorm1d(tab_dim)
        self.classifier = nn.Sequential(
            nn.Flatten(),                          # flatten image feats
            # image feature dim = base_model.fc.in_features
            # + tabular dim
            nn.Linear(base_model.fc.in_features + tab_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, img, tab):
        x = self.backbone(img)       # [B,2048,1,1]
        x = torch.flatten(x, 1)      # [B,2048]
        tab = self.tab_bn(tab)       # normalize tabular
        x = torch.cat([x, tab], dim=1)
        return self.classifier(x)


In [8]:
# FINE‑TUNING CLASS 
class FineTuneCNN:
    def __init__(self, train_ds, valid_ds, base_model, batch_size=32, freeze_ratio=1.0, test_ds=None):
        self.model = base_model
        # freeze parameters
        if freeze_ratio >= 1.0:
            for p in self.model.parameters():
                p.requires_grad = False
        else:
            total_p = sum(1 for _ in self.model.parameters())
            to_freeze = int(total_p * freeze_ratio)
            cnt = 0
            for p in self.model.parameters():
                p.requires_grad = False
                cnt += 1
                if cnt >= to_freeze:
                    break
        # ensure final layers are trainable
        for p in self.model.classifier.parameters():
            p.requires_grad = True

        # override the default loaders
        self.train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        self.valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
        self.test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False) if test_ds else None

    def run_training(self, num_epochs=10, learning_rate=1e-3, weight_decay_val=0):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(),
                               lr=learning_rate,
                               weight_decay=weight_decay_val)

        for epoch in range(1, num_epochs+1):
            # --- train ---
            self.model.train()
            running_correct = 0
            running_total   = 0
            for img, tab, lbl in self.train_loader:
                img, tab, lbl = img.to(device), tab.to(device), lbl.to(device)
                optimizer.zero_grad()
                out = self.model(img, tab)
                loss = criterion(out, lbl)
                loss.backward()
                optimizer.step()
                pred = out.argmax(dim=1)
                running_correct += (pred == lbl).sum().item()
                running_total   += lbl.size(0)
            train_acc = 100 * running_correct / running_total
            print(f"Epoch {epoch} — Train Acc: {train_acc:.2f}%")
            wandb.log({"epoch":epoch, "train_acc":train_acc})

            # --- validate ---
            self.model.eval()
            val_corr = 0
            val_tot  = 0
            val_loss = 0.0
            with torch.no_grad():
                for img, tab, lbl in self.valid_loader:
                    img, tab, lbl = img.to(device), tab.to(device), lbl.to(device)
                    out = self.model(img, tab)
                    l  = criterion(out, lbl)
                    pred = out.argmax(dim=1)
                    val_corr += (pred == lbl).sum().item()
                    val_tot  += lbl.size(0)
                    val_loss  = l.item()
            val_acc = 100 * val_corr / val_tot
            print(f"Epoch {epoch} — Val   Acc: {val_acc:.2f}%")
            wandb.log({"validation_accuracy": val_acc, "validation_loss": val_loss})

    def evaluate_test(self):
        if self.test_loader is None:
            print("No test set.")
            return
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        test_corr = 0
        test_tot  = 0
        with torch.no_grad():
            for img, tab, lbl in self.test_loader:
                img, tab, lbl = img.to(device), tab.to(device), lbl.to(device)
                out = self.model(img, tab)
                pred = out.argmax(dim=1)
                test_corr += (pred == lbl).sum().item()
                test_tot  += lbl.size(0)
        test_acc = 100 * test_corr / test_tot
        print(f"Test Accuracy: {test_acc:.2f}%")
        wandb.log({"test_accuracy": test_acc})



In [9]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [10]:
wandb.login(key='1df7a902fa4a610500b8e79e21818419d5facdbb')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m018 (ma23m018-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
#  WANDB SWEEP SETUP

sweep_config = {
    'method': 'bayes',
    'metric': {'goal': 'maximize','name': 'validation_accuracy'},
    'parameters': {
        'learning_rate': {'values':[1e-3,1e-4]},
        'freeze_ratio':  {'values':[0.2,0.6,0.9]},
        'l2_reg':        {'values':[0,5e-4,5e-2]},
        'batch_size':    {'values':[32,64]},
        'epochs':        {'values':[5,10]}
    }
}
sweep_id = wandb.sweep(sweep_config, entity= "ma23m018-indian-institute-of-technology-madras", project="mtech_project2")

Create sweep with ID: uxd36tyz
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/mtech_project2/sweeps/uxd36tyz


In [12]:
#sweep_id = wandb.sweep(sweep_config, entity= "ma23m018-indian-institute-of-technology-madras", project="mtech_project1")

Create sweep with ID: nlvlh2fk
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/mtech_project1/sweeps/nlvlh2fk


In [12]:
def main():
    with wandb.init() as run:
        cfg = wandb.config
        run.name = f"bs{cfg.batch_size}_ep{cfg.epochs}_lr{cfg.learning_rate}_fr{cfg.freeze_ratio}"

        # build your ResNet+tabular model
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        model = ResNetWithTabular(
            base_model=base,
            tab_dim=feature_dim,
            num_classes=10
        )

        # initialize FineTuneCNN with test_ds=None
        finetuner = FineTuneCNN(
            train_ds=train_ds,
            valid_ds=val_ds,
            base_model=model,
            batch_size=cfg.batch_size,
            freeze_ratio=cfg.freeze_ratio,
            test_ds=None    # do not use the test folder here
        )

        # only train+validate
        finetuner.run_training(
            num_epochs=cfg.epochs,
            learning_rate=cfg.learning_rate,
            weight_decay_val=cfg.l2_reg
        )
        


# 
# each run will sample a new hyperparameter set and only log validation_accuracy
wandb.agent(sweep_id, function=main, count=20)

wandb: Agent Starting Run: 41rxpjk5 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	freeze_ratio: 0.6
wandb: 	l2_reg: 0.0005
wandb: 	learning_rate: 0.001
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 189MB/s] 


Epoch 1 — Train Acc: 55.33%
Epoch 1 — Val   Acc: 58.98%
Epoch 2 — Train Acc: 61.58%
Epoch 2 — Val   Acc: 57.78%
Epoch 3 — Train Acc: 64.34%
Epoch 3 — Val   Acc: 64.23%
Epoch 4 — Train Acc: 66.96%
Epoch 4 — Val   Acc: 64.41%
Epoch 5 — Train Acc: 69.72%
Epoch 5 — Val   Acc: 63.54%


epoch,▁▃▅▆█
train_acc,▁▄▅▇█
validation_accuracy,▂▁██▇
validation_loss,▇█▃▁▃
epoch,5
train_acc,69.72486
validation_accuracy,63.53591
validation_loss,0.90066


wandb: Agent Starting Run: flu04a99 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	freeze_ratio: 0.6
wandb: 	l2_reg: 0.0005
wandb: 	learning_rate: 0.0001


...

Epoch 1 — Train Acc: 58.36%
Epoch 1 — Val   Acc: 62.98%
Epoch 2 — Train Acc: 68.05%
Epoch 2 — Val   Acc: 68.05%
Epoch 3 — Train Acc: 75.24%
Epoch 3 — Val   Acc: 69.38%
Epoch 4 — Train Acc: 81.71%
Epoch 4 — Val   Acc: 65.93%
Epoch 5 — Train Acc: 87.16%
Epoch 5 — Val   Acc: 71.32%


epoch,▁▃▅▆█
train_acc,▁▃▅▇█
validation_accuracy,▁▅▆▃█
validation_loss,▇▄▄█▁
epoch,5
train_acc,87.15864
validation_accuracy,71.31676
validation_loss,0.75135


wandb: Agent Starting Run: htpzyaf7 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	freeze_ratio: 0.2
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0001


Epoch 1 — Train Acc: 59.61%
Epoch 1 — Val   Acc: 64.96%
Epoch 2 — Train Acc: 71.46%
Epoch 2 — Val   Acc: 68.19%
Epoch 3 — Train Acc: 79.58%
Epoch 3 — Val   Acc: 69.94%
Epoch 4 — Train Acc: 86.51%
Epoch 4 — Val   Acc: 69.24%
Epoch 5 — Train Acc: 92.07%
Epoch 5 — Val   Acc: 70.03%
Epoch 6 — Train Acc: 94.76%
Epoch 6 — Val   Acc: 70.86%
Epoch 7 — Train Acc: 95.69%
Epoch 7 — Val   Acc: 70.95%
Epoch 8 — Train Acc: 96.88%
Epoch 8 — Val   Acc: 69.84%
Epoch 9 — Train Acc: 97.67%
Epoch 9 — Val   Acc: 69.66%
Epoch 10 — Train Acc: 97.30%
Epoch 10 — Val   Acc: 68.88%


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▅▆▇▇████
validation_accuracy,▁▅▇▆▇██▇▆▆
validation_loss,▃▁▁▂▄▅▄▅█▃
epoch,10
train_acc,97.29979
validation_accuracy,68.87661
validation_loss,0.92299
